In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset

In [2]:
# load array
Data = load('Data_09.npy')
# print the array

In [3]:
Data=np.nan_to_num(Data)

In [4]:
Xdata=[]
Ydata=[]
Xtrain_data=[]
Ytrain_data=[]
Xtest_data=[]
Ytest_data=[]

for idx in range(len(Data)//39):
#     print(np.isinf(Data[39*idx:39*(idx)+38][:,:].tolist()).any())
    if(np.isinf(Data[39*idx:39*(idx)+39][:,:].tolist()).any()):
        print(np.isinf(Data[39*idx:39*(idx)+39][:,:].tolist()).any())
        continue
    if idx< 3735:
        Xtrain_data.append(Data[39*idx:39*(idx)+39][:,:].tolist())
        Ytrain_data.append(Data[39*idx:39*(idx)+39][:,-1].tolist())
    else:
        Xtest_data.append(Data[39*idx:39*(idx)+39][:,:].tolist())
        Ytest_data.append(Data[39*idx:39*(idx)+39][:,-1].tolist())

Xtrain_data=np.vstack(Xtrain_data)
Ytrain_data=np.vstack(Ytrain_data)

Xtrain_data=torch.FloatTensor(Xtrain_data)
Ytrain_data=torch.IntTensor(Ytrain_data)
Ytrain_data=Ytrain_data.view(-1)


Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)
Xtest_data=torch.FloatTensor(Xtest_data)
Ytest_data=torch.IntTensor(Ytest_data)
Ytest_data=Ytest_data.view(-1)

In [5]:
Xtrain_data.size()

torch.Size([145665, 7])

In [6]:
Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()

Ydata=(torch.stack([Ydata>0,Ydata==0, Ydata<0],axis=0)).float()

In [7]:
Ytrain_data=Ytrain_data.T
Ytest_data=Ytest_data.T

In [8]:
Xtrain_data.size()

torch.Size([145665, 7])

In [9]:
Ytrain_data

tensor([1, 1, 1,  ..., 2, 2, 1])

In [10]:
Xtrain_data

tensor([[1.2950e+04, 1.7992e-02, 1.9043e-02,  ..., 5.0380e+04, 4.6628e+04,
         0.0000e+00],
        [1.2950e+04, 2.3718e-02, 3.0888e-02,  ..., 6.4586e+04, 5.3998e+04,
         0.0000e+00],
        [1.2950e+04, 4.1475e-02, 4.7411e-02,  ..., 8.1665e+04, 6.4157e+04,
         0.0000e+00],
        ...,
        [9.5000e+03, 1.1409e-02, 4.4211e-02,  ..., 7.1436e+04, 2.3673e+04,
         7.0100e+02],
        [9.5000e+03, 6.9253e-03, 4.4211e-02,  ..., 6.9510e+04, 1.8099e+04,
         1.1000e+01],
        [9.5000e+03, 9.9207e-03, 4.4211e-02,  ..., 6.9436e+04, 1.9535e+04,
         0.0000e+00]])

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # 데이터셋을 bsz 파트들로 나눕니다.
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)

# batch_size = 25
# eval_batch_size = 25

# Xtrain_data = batchify(Xtrain_data, batch_size)
# Xtest_data = batchify(Xtest_data, eval_batch_size)
# Ytrain_data = batchify(Ytrain_data, batch_size)
# Ytest_data = batchify(Ytest_data, eval_batch_size)

In [12]:
bptt = 39
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [13]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

INPUT_SIZE=7
EMB_SIZE=512
HIDDEN_SIZE=512
NUM_LAYERS=6
PROJ_SIZE=3
BATCH_SIZE=10

In [14]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import numpy as np

def batch_matmul(seq, weight, nonlinearity=''):
    s = None
    for i in range(seq.size(0)):
        _s = torch.mm(seq[i], weight)
        if(nonlinearity=='tanh'):
            _s = torch.tanh(_s)
        _s = _s.unsqueeze(0)
        if(s is None):
            s = _s
        else:
            s = torch.cat((s,_s),0)
    return s.squeeze()

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers,proj_size, dropout=0.5, tie_weights=False, attention=True,
                 attention_width=3, cuda=True):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
#         self.encoder = nn.Embedding(ntoken, ninp)
        self.encoder = nn.Linear(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError("""An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        if attention:
            self.decoder = nn.Linear(nhid, ntoken)
        else:
            self.decoder = nn.Linear(nhid, ntoken)
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.softmax = nn.Softmax()
        if attention:
            self.AttentionLayer = AttentionLayer(cuda,nhid)
        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers
        self.attention = attention
        self.attention_width = attention_width
        self.generator = nn.Linear(nhid, proj_size)
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input):
        #print("input size:",input.size())
#         emb = self.drop(self.encoder(input))
        emb=self.encoder(input)
        #print("emb size:",emb.size())
        output, _ = self.rnn(emb)
        #print("rnn output",output.size())
        if self.attention:
            output = self.AttentionLayer.forward(output, self.attention_width)
        return self.generator(output)
        #
#         output = self.drop(output)
#         decoded = self.decoder(output.view(output.size(0) * output.size(1), output.size(2)))
#         return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (Variable(weight.new(self.nlayers, bsz, self.nhid).zero_()),
                    Variable(weight.new(self.nlayers, bsz, self.nhid).zero_()))
        else:
            return Variable(weight.new(self.nlayers, bsz, self.nhid).zero_())


class AttentionLayer(nn.Module):
    """Implements an Attention Layer"""

    def __init__(self, cuda, nhid):
        super(AttentionLayer, self).__init__()
        self.nhid = nhid
        self.weight_W = nn.Parameter(torch.Tensor(nhid,nhid))
        self.weight_proj = nn.Parameter(torch.Tensor(nhid, 1))
        self.softmax = nn.Softmax()
        self.weight_W.data.uniform_(-0.1, 0.1)
        self.weight_proj.data.uniform_(-0.1,0.1)
        self.cuda = cuda

    def forward(self, inputs, attention_width=3):
        results = None
        for i in range(inputs.size(0)):
            if(i<attention_width):
                output = inputs[i]
                output = output.unsqueeze(0)

            else:
                lb = i - attention_width
                if(lb<0):
                    lb = 0
                selector = torch.from_numpy(np.array(np.arange(lb, i)))
                if self.cuda:
                    selector = Variable(selector).cuda()
                else:
                    selector = Variable(selector)
                vec = torch.index_select(inputs, 0, selector)
                u = batch_matmul(vec, self.weight_W, nonlinearity='tanh')
                a = batch_matmul(u, self.weight_proj)
                a = self.softmax(a)
                output = None
                for i in range(vec.size(0)):
                    h_i = vec[i]
                    a_i = a[i].unsqueeze(1).expand_as(h_i)
                    h_i = a_i * h_i
                    h_i = h_i.unsqueeze(0)
                    if(output is None):
                        output = h_i
                    else:
                        output = torch.cat((output,h_i),0)
                output = torch.sum(output,0)
                output = output.unsqueeze(0)

            if(results is None):
                results = output

            else:
                results = torch.cat((results,output),0)

        return results

In [16]:
torch.manual_seed(0)



#lstm=Seq2SeqLSTM(input_size=INPUT_SIZE,hidden_size=HIDDEN_SIZE,num_layers=NUM_LAYERS,emb_size=EMB_SIZE,proj_size=PROJ_SIZE)

#lstm = RNNmodel(rnn_type='LSTM',ntoken=INPUT_SIZE,ninp=INPUT_SIZE,nhid=HIDDEN_SIZE,nlayers=NUM_LAYERS,emb_size=EMB_SIZE,proj_size=PROJ_SIZE)
lstm = RNNModel(rnn_type='LSTM',ntoken=INPUT_SIZE,ninp=EMB_SIZE,nhid=HIDDEN_SIZE,nlayers=NUM_LAYERS,proj_size=PROJ_SIZE,
                attention_width=38)
# for p in lstm.parameters():
#     if p.dim() > 1:
#         nn.init.xavier_uniform_(p)

lstm = lstm.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-9)

In [17]:
(Ytest_data==2).all()

tensor(False)

In [18]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0

    for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
#            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)
#         src = src.to(DEVICE)
#         tgt = tgt.to(DEVICE)
        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE)
        tgt=batchify(targets,BATCH_SIZE)
        src=src[:-1]
        tgt=tgt[1:]
        logits = model(src)

        optimizer.zero_grad()

        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
    return losses / Xtrain_data.size(0)


def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    with torch.no_grad():
        for i in range(0, Xtest_data.size(0) - 1, bptt):

            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
#            print(data)
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE)
            tgt=batchify(targets,BATCH_SIZE)
#             print(src.size())
#             print(tgt.size())
            logits = model(src)
#             print(logits)
#             print(logits.size())

            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
            if(loss.isnan()):
                print(src,tgt_input)
                break
#            print(loss)
            losses += loss.item()
#             output_flat = logits.view(-1)
            _,predicted=torch.max(logits,-1)
#             print(predicted)
#             print(logits.size())
#             print(tgt_out.size())
            
            correct += (tgt.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt.squeeze()).sum().item()
            tot1+=(1== tgt.squeeze()).sum().item()
            tot2+=(2== tgt.squeeze()).sum().item()
            correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
            
    print(total)
    print(correct)
    print("Acc:",correct/total*100)            
    print("Acc0:",correct0/tot0*100,correct0,tot0)     
    print("Acc1:",correct1/tot1*100,correct1,tot1)     
    print("Acc2:",correct2/tot2*100,correct2,tot2)     
    return losses / Xtest_data.size(0)

In [19]:
evaluate(lstm)

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


288740
55129
Acc: 19.092955600193946
Acc0: 100.0 55129 55129
Acc1: 0.0 0 189131
Acc2: 0.0 0 44480


0.02856428215783099

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 1000
best_val_loss=100000000
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(lstm, optimizer)
    end_time = timer()
    val_loss = evaluate(lstm)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = lstm
PATH='best_model_alstm_seq'
torch.save(best_model.state_dict(), PATH)